In [1]:
# Reference : https://github.com/golbin/TensorFlow-Tutorials
import tensorflow as tf
import numpy as np

In [2]:
# [털, 날개]
x_data = np.array([[0,0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

# tf.name_scope

tf.name_scope로 묶은 블럭은 텐서보드에서 한 레이어 안에서 표현됨

In [28]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1.0, 1.0), name='W1')
    b1 = tf.Variable(tf.zeros([10]), name='b1')
    L1 = tf.add(tf.matmul(X, W1), b1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1.0, 1.0), name='W2')
    b2 = tf.Variable(tf.zeros([20]), name='b2')
    L2 = tf.add(tf.matmul(L1, W2), b2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1.0, 1.0), name='W3')
    b3 = tf.Variable(tf.zeros([3]), name='b3')
    model = tf.add(tf.matmul(L2, W3), b3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # tf.summary.scalar를 이용해서 수집하고 싶은 값을 지정할 수 있음
    tf.summary.scalar('cost', cost)

In [29]:
# 모델 학습

In [34]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model/6')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 텐서 정보 수집

tensorboard --logdir=./logs 으로 실행하여 확인 가능

In [37]:
merged = tf.summary.merge_all()
# 그래프와 텐서값들을 저장할 디렉토리 설정
writer = tf.summary.FileWriter('./logs/6', sess.graph)

print(merged)

Tensor("Merge_4/MergeSummary:0", shape=(), dtype=string)


# 최적화

In [38]:
for step in range(100):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    print('Step : %d ' % sess.run(global_step))
    print('Cost : %.3f ' % sess.run(cost, feed_dict={X: x_data, Y:y_data}))
    
    # 텐서를 수집하고 저장함
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/6/ckpt', global_step=global_step)

Step : 1 
Cost : 0.738 
Step : 2 
Cost : 0.591 
Step : 3 
Cost : 0.470 
Step : 4 
Cost : 0.369 
Step : 5 
Cost : 0.286 
Step : 6 
Cost : 0.218 
Step : 7 
Cost : 0.164 
Step : 8 
Cost : 0.123 
Step : 9 
Cost : 0.091 
Step : 10 
Cost : 0.068 
Step : 11 
Cost : 0.051 
Step : 12 
Cost : 0.038 
Step : 13 
Cost : 0.029 
Step : 14 
Cost : 0.022 
Step : 15 
Cost : 0.017 
Step : 16 
Cost : 0.014 
Step : 17 
Cost : 0.011 
Step : 18 
Cost : 0.009 
Step : 19 
Cost : 0.007 
Step : 20 
Cost : 0.006 
Step : 21 
Cost : 0.005 
Step : 22 
Cost : 0.005 
Step : 23 
Cost : 0.004 
Step : 24 
Cost : 0.004 
Step : 25 
Cost : 0.003 
Step : 26 
Cost : 0.003 
Step : 27 
Cost : 0.003 
Step : 28 
Cost : 0.002 
Step : 29 
Cost : 0.002 
Step : 30 
Cost : 0.002 
Step : 31 
Cost : 0.002 
Step : 32 
Cost : 0.002 
Step : 33 
Cost : 0.001 
Step : 34 
Cost : 0.001 
Step : 35 
Cost : 0.001 
Step : 36 
Cost : 0.001 
Step : 37 
Cost : 0.001 
Step : 38 
Cost : 0.001 
Step : 39 
Cost : 0.001 
Step : 40 
Cost : 0.001 
Step : 41

'./model/6/ckpt-100'

# 결과 확인

In [39]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' % sess.run(accuracy*100, feed_dict = {X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도 : 100.00
